In [ ]:
## 载入QA嵌入向量数据
import pickle
import numpy as np
model_list=["mini_rag","qag_llm_7B","qwen2.5-7b-instruct-1m",
            "qag_llm_14B","qwen_max","qwen_plus","deepseek-R1",
            "deepseek-v3","deepseek-r1-distill-qwen-7b",
            "deepseek-r1-distill-qwen-14b","llama3.3-70b-instruct","qwq_32b"]

qa_embs=[[] for _ in model_list]
with open('qa_embs.pkl', 'rb') as file:
    qa_emb = pickle.load(file)
with open('qa_embs_1.pkl', 'rb') as file:
    qa_emb_1 = pickle.load(file)
qa_embs[0]=qa_emb[0]
qa_embs[1]=qa_emb[1]
qa_embs[2]=qa_emb_1[6]
qa_embs[3]=qa_emb_1[0]
qa_embs[4]=qa_emb[2]
qa_embs[5]=qa_emb[3]
qa_embs[6]=qa_emb_1[1]
qa_embs[7]=qa_emb_1[4]
qa_embs[8]=qa_emb_1[3]
qa_embs[9]=qa_emb_1[2]
qa_embs[10]=qa_emb_1[5]
qa_embs[11]=qa_emb_1[7]

In [ ]:
## 载入文章分块嵌入向量数据
with open('article_embs.pkl', 'rb') as file:
    article_emb = pickle.load(file)

In [ ]:
## 打分+存储结果
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

import pandas as pd
relavance=[[] for _ in qa_embs]
for i in range(len(qa_embs)):
    scores=[]
    for j in range(30):
        ##qa
        for k in qa_embs[i][j]:
            q=k[0]
            a=k[1]
            s=0
            for p in article_emb[j]:
                s=s+q.dot(p)+a.dot(p)
            s=s/len(article_emb[j])
            scores.append(s)
    relavance[i].append(scores)

completeness=[[] for _ in qa_embs]
QA_filds=[[] for _ in qa_embs]
for i in range(30):
    for j in range(len(qa_embs)):
        q_filds=[]
        a_filds=[]

        for k in qa_embs[j][i]:
            q_fild=[]
            a_fild=[]
            for p in article_emb[i]:
                q_fild.append(p.dot(k[0]))
                a_fild.append(p.dot(k[1]))
            q_fild_ = np.array(q_fild)
            a_fild_ = np.array(a_fild)
            # 计算逐元素乘积
            qam = np.multiply(q_fild_, a_fild_)
            # ((问题与答案的相似度) * [Σ (答案分块相似度 * 问题分块相似度)] / [Σ 问题分块相似度])^0.5
            score=(k[0].dot(k[1])*sum(qam)/sum(q_fild))**0.5
            q_filds.append(q_fild)
            a_filds.append(a_fild)
            print(q_filds,a_filds)
            if score>=1:
                completeness[j].append(1)
            else:
                completeness[j].append(score)
        QA_filds[j].append({'q_filds':q_filds,'a_filds':a_filds})

##准确性
accuracy=[[] for _ in qa_embs]
for i in range(30):
    for k in range(len(qa_embs)):
        for j in range(len(qa_embs[k][i])):
            q_filds_reverse=QA_filds[k][i]['q_filds'][j].copy()
            q_filds_reverse.sort(reverse=True)
            a_filds_reverse=QA_filds[k][i]['a_filds'][j].copy()
            a_filds_reverse.sort(reverse=True)
            q_top5_value=q_filds_reverse[4]
            a_top5_value=a_filds_reverse[4]
            rel=[]
            ref=[]
            n_cor=0
            for p in range(len(QA_filds[k][i]['q_filds'][j])):
                if QA_filds[k][i]['q_filds'][j][p]>0.6*0.5+0.5*q_top5_value:
                    rel.append(p)
                if QA_filds[k][i]['a_filds'][j][p]>0.7*0.5+0.5*((q_top5_value*a_top5_value))**0.5:
                    ref.append(p)
            for p in ref:
                if p in rel:
                    n_cor=n_cor+1
            score=n_cor/max(1,len(rel))
            accuracy[k].append(min(1,score))
relavance_s=[]
completeness_s=[]
accuracy_s=[]
for i in range(len(qa_embs)):

    relavance_s.append(sum([k for j in relavance[i] for k in j])/len([k for j in relavance[i] for k in j]))
    completeness_s.append(sum(completeness[i])/len((completeness[i])))
    accuracy_s.append(sum(accuracy[i])/len(accuracy[i]))


relavance_s[0]=relavance_s[0]*30/29
relavance_s=np.array(relavance_s)
relavance_s=(sigmoid(relavance_s/2)*relavance_s/2)**0.5
relavance_s=relavance_s.tolist()
data=pd.DataFrame({"model":model_list,"relavance":relavance_s,"completeness":completeness_s,"accuracy":accuracy_s})
data.to_csv('test.csv')